In [1]:
import os
# python2 print os.getcwd()
# python3 print(os.getcwd))
os.getcwd()

'/Users/CreativeMindIV/Google Drive/Education/Statistical Learning - 2016/Kaggle - Digits'

In [2]:
# mac os.chdir('/Users/CreativeMindIV/Google Drive/Education/Statistical Learning - 2016/Kaggle - Digits')
# Windows os.getcwd().replace('\\','/')
os.chdir('/Users/CreativeMindIV/Google Drive/Education/Statistical Learning - 2016/Kaggle - Digits')

In [3]:
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.utils import np_utils
from keras.layers.core import Dense, Activation, Dropout
from keras.constraints import maxnorm
from keras.optimizers import SGD

Using TensorFlow backend.


In [4]:
# Read data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [47]:
# indexing and selecting data with Pandas
# define Data.Frame
raw = {'Neighborhood': ['Crest', 'Oak', 'Height', 'Crescent', 'Arbor'],
           'Beds': [3, 4, 3, 3, 4],
           'Baths': [3, 4, 2, 3, 3],
           'Size': [2000, 2500, 2100, 2000, 2300]
           }
df = pd.DataFrame(raw, 
        columns = ['Neighborhood', 'Beds', 'Baths', 'Size'], 
        index=['Blakely', 'Laidlaw', 'Springfield', 'Kaiser','Bethany'])
df

,Neighborhood,Beds,Baths,Size
Blakely,Crest,3,3,2000
Laidlaw,Oak,4,4,2500
Springfield,Height,3,2,2100
Kaiser,Crescent,3,3,2000
Bethany,Arbor,4,3,2300


In [79]:
# SELECT COLUMN(S) BY INDEX LABEL
df.loc[:,'Beds']

Blakely        3
Laidlaw        4
Springfield    3
Kaiser         3
Bethany        4
Name: Beds, dtype: int64

In [74]:
# SELECT ROW(S) BY INDEX LABEL
df.loc['Blakely', :]

Neighborhood    Crest
Beds                3
Baths               3
Size             2000
Name: Blakely, dtype: object

In [76]:
# SELECT ROW(S) BY POSITION INDEX
df.iloc[0,:]

Neighborhood    Crest
Beds                3
Baths               3
Size             2000
Name: Blakely, dtype: object

In [78]:
# SELECT COLUMN(S) by POSITION INDEX
df.iloc[:,1]

Blakely        3
Laidlaw        4
Springfield    3
Kaiser         3
Bethany        4
Name: Beds, dtype: int64

In [72]:
# SELECT COLUMN(S) USING GENERAL INDEXING
df.ix[:,1]

Blakely        3
Laidlaw        4
Springfield    3
Kaiser         3
Bethany        4
Name: Beds, dtype: int64

In [67]:
# df.ix[ , ].values convert into array
df.ix[:,1].values

array([3, 4, 3, 3, 4])

In [80]:
df.ix[:,1].values.astype('float32')

array([ 3.,  4.,  3.,  3.,  4.], dtype=float32)

In [32]:
labels = train.ix[:,0].values.astype('int32')
X_train = (train.ix[:,1:].values).astype('float32')
X_test = (test.values).astype('float32')

In [33]:
# convert list of labels to binary class matrix
y_train = np_utils.to_categorical(labels) 

In [34]:
# pre-processing: divide by max and substract mean
scale = np.max(X_train)
X_train /= scale
X_test /= scale

mean = np.std(X_train)
X_train -= mean
X_test -= mean

In [35]:
X_train[0,:]

array([-0.30853882, -0.30853882, -0.30853882, -0.30853882, -0.30853882,
       -0.30853882, -0.30853882, -0.30853882, -0.30853882, -0.30853882,
       -0.30853882, -0.30853882, -0.30853882, -0.30853882, -0.30853882,
       -0.30853882, -0.30853882, -0.30853882, -0.30853882, -0.30853882,
       -0.30853882, -0.30853882, -0.30853882, -0.30853882, -0.30853882,
       -0.30853882, -0.30853882, -0.30853882, -0.30853882, -0.30853882,
       -0.30853882, -0.30853882, -0.30853882, -0.30853882, -0.30853882,
       -0.30853882, -0.30853882, -0.30853882, -0.30853882, -0.30853882,
       -0.30853882, -0.30853882, -0.30853882, -0.30853882, -0.30853882,
       -0.30853882, -0.30853882, -0.30853882, -0.30853882, -0.30853882,
       -0.30853882, -0.30853882, -0.30853882, -0.30853882, -0.30853882,
       -0.30853882, -0.30853882, -0.30853882, -0.30853882, -0.30853882,
       -0.30853882, -0.30853882, -0.30853882, -0.30853882, -0.30853882,
       -0.30853882, -0.30853882, -0.30853882, -0.30853882, -0.30

In [36]:
input_dim = X_train.shape[1]
nb_classes = y_train.shape[1]

In [84]:
# Here's a Deep Dumb MLP (DDMLP)
# dropout is a regularization method for neural net
# The dropout rate of 20% or one in 5 inputs will be randomly excluded from each update cycle.
# additionally, we impose constraint on weight for each hidden layer
# to make sure that maximum norm of weight does not exceed value of 3
# this is done W_constraint = maxnorm(3)

model = Sequential()
model.add(Dense(128, input_dim=input_dim, init='normal', W_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.20))
model.add(Dense(128, init='normal', W_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.20))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [92]:
# we'll use categorical crossentropy for the loss, and sgd as the optimizer
sgd = SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=False)
# model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
model.compile(loss='categorical_crossentropy', optimizer='sgd')

print("Training...")
model.fit(X_train, y_train, nb_epoch=10, batch_size=100, validation_split=0.10, show_accuracy=True, verbose=2)

print("Generating test predictions...")
preds = model.predict_classes(X_test, verbose=0)

Training...
Train on 37800 samples, validate on 4200 samples
Epoch 1/10
4s - loss: 0.0751 - val_loss: 0.0897
Epoch 2/10
3s - loss: 0.0740 - val_loss: 0.0905
Epoch 3/10
3s - loss: 0.0742 - val_loss: 0.0896
Epoch 4/10
4s - loss: 0.0732 - val_loss: 0.0886
Epoch 5/10
3s - loss: 0.0745 - val_loss: 0.0897
Epoch 6/10
3s - loss: 0.0735 - val_loss: 0.0897
Epoch 7/10
3s - loss: 0.0732 - val_loss: 0.0892
Epoch 8/10
4s - loss: 0.0726 - val_loss: 0.0878
Epoch 9/10
3s - loss: 0.0719 - val_loss: 0.0883
Epoch 10/10
3s - loss: 0.0710 - val_loss: 0.0882
Generating test predictions...


In [93]:
def write_preds(preds, fname):
    pd.DataFrame({"ImageId": list(range(1,len(preds)+1)), "Label": preds}).to_csv(fname, index=False, header=True)

write_preds(preds, "keras-mlp.csv")
# rank 998 (0.96529), dropout = 0.15
# rank 1007 (0.96529) dropout = 0.20, init='normal', W_constraint = maxnorm(3)
# add sgd, lr=0.01, momentum=0.8, Dense(128) epoch=10 rank 703 (0.9730)
# lr=0.01, momentum=0.9 Dense(128) epoch=10 rank 671 (0.97429)
# lr=0.01, momentum=0.9 Dense(128*2) epoch=10... rank 671 (0.97429)
# lr=0.1, momentum=0.9 Dense(128) epoch = 10...rank 671 (0.97429)
# lr=0.1, momentum=0.9 Dense(128) epoch = 10, nesterov=True
# same but Dense = 512, rank 674 (0.97429)
# back to Dense = 128, increase validation split from 0.1 to 0.3
# back to validation split 0.15.  Increase batch size from 16 to 100. 0.97029
# reset nesterov = False. 0.97043